In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
X_train = np.random.randint(3, high=32, size=(11000,16))
Y_train = np.sort(X_train)
X_target = np.copy(Y_train)
X_target[:,1:] = X_target[:,:-1]
X_target[:,0] = 1
X_train = np.concatenate([X_train, X_target], axis=-1)
X_train,X_test = X_train[1000:],X_train[:1000]
Y_train,Y_test = Y_train[1000:],Y_train[:1000]
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((10000, 32), (10000, 16), (1000, 32), (1000, 16))

In [3]:
data_input_dim = max(np.max(X_train), np.max(X_test)) + 1
data_input_dim

32

In [4]:
data_output_dim = max(np.max(Y_train), np.max(Y_test)) + 1
data_output_dim

32

In [5]:
class DNCCell(keras.layers.Layer):
    
    def __init__(self,
                 output_size,
                 word_size,
                 memory_size,
                 register_size=2,
                 write_head_count=2,
                 read_head_count=2,
                 enable_temporal=True,
                 enable_controller_output_resize=True,
                 enable_final_bias=True,
                 bypass_dropout_factor=None,
                 **kwargs):
        self.output_size = output_size
        self.word_size = word_size
        self.memory_size = memory_size
        self.register_size = register_size
        self.write_head_count = write_head_count
        self.read_head_count = read_head_count
        self.enable_temporal = enable_temporal
        self.enable_controller_output_resize = enable_controller_output_resize
        self.enable_final_bias = enable_final_bias
        self.bypass_dropout_factor = bypass_dropout_factor
        self.state_size = (output_size,)
        self.state_size += (register_size * word_size,)
        self.state_size += (register_size * word_size,)
        self.state_size += (memory_size * word_size,)
        self.state_size += (read_head_count * memory_size,)
        self.state_size += (read_head_count * word_size,)
        self.state_size += (write_head_count * memory_size,)
        self.state_size += (memory_size,)
        if enable_temporal:
            self.state_size += (write_head_count * memory_size,)
            self.state_size += (write_head_count * memory_size**2,)
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        read_vec_size = self.word_size * self.read_head_count
        controller_input_size = input_shape[-1]
        controller_input_size += read_vec_size
        controller_input_size += self.register_size*self.word_size
        controller_kernel_size = self.register_size*self.word_size*4
        controller_hidden_size = self.register_size*self.word_size
        interface_vec_size = read_vec_size
        interface_vec_size += self.read_head_count*2
        if self.enable_temporal:
            interface_vec_size += self.read_head_count*3
        interface_vec_size += self.write_head_count*self.word_size*3
        interface_vec_size += self.write_head_count*3
        self.kernel_controller_hidden = self.add_weight(
            shape=(controller_input_size, controller_kernel_size),
            initializer='glorot_normal', name='kernel_controller_hidden')
        self.bias_controller_hidden = self.add_weight(
            shape=(controller_kernel_size,),
            initializer='zeros', name='bias_controller_hidden')
        self.kernel_controller_output = self.add_weight(
            shape=(controller_hidden_size, self.output_size + interface_vec_size),
            initializer='glorot_normal', name='kernel_controller_output')
        self.kernel_read_vec_to_output = self.add_weight(
            shape=(read_vec_size, self.output_size),
            initializer='glorot_normal', name='kernel_read_vec_to_output')
        if self.enable_controller_output_resize:
            self.kernel_controller_output_resize = self.add_weight(
                shape=(self.output_size, self.output_size),
                initializer='glorot_normal', name='kernel_controller_output_resize')
        if self.enable_final_bias:
            self.bias_final_output = self.add_weight(
                shape=(self.output_size,),
                initializer='zeros', name='bias_final_output')
        super().build(input_shape)
    
    def call(self, inputs, states, training=None):
        
        def oneplus(x):
            return K.softplus(x) + 1.
        
        def similarity(m, k, b):
            dot = K.batch_dot(k, m, axes=2)
            m_len = K.sqrt(K.sum(K.square(m), axis=-1))
            k_len = K.sqrt(K.sum(K.square(k), axis=-1))
            mk_len = K.expand_dims(k_len, axis=-1) @ K.expand_dims(m_len, axis=-2)
            mk_len = K.switch(K.not_equal(mk_len, 0.), mk_len, K.ones_like(mk_len))
            cos_sim = dot / mk_len
            return K.softmax(cos_sim * K.expand_dims(b))
        
        def batch_invert_permutation(permutations):
            # from https://github.com/deepmind/dnc/blob/master/dnc/util.py
            perm = tf.cast(permutations, tf.float32)
            dim = int(perm.get_shape()[-1])
            size = tf.cast(tf.shape(perm)[0], tf.float32)
            delta = tf.cast(tf.shape(perm)[-1], tf.float32)
            rg = tf.range(0, size * delta, delta, dtype=tf.float32)
            rg = tf.expand_dims(rg, 1)
            rg = tf.tile(rg, [1, dim])
            perm = tf.add(perm, rg)
            flat = tf.reshape(perm, [-1])
            perm = tf.invert_permutation(tf.cast(flat, tf.int32))
            perm = tf.reshape(perm, [-1, dim])
            return tf.subtract(perm, tf.cast(rg, tf.int32))

        def batch_gather(values, indices):
            # from https://github.com/deepmind/dnc/blob/master/dnc/util.py
            idx = tf.expand_dims(indices, -1)
            size = tf.shape(indices)[0]
            rg = tf.range(size, dtype=tf.int32)
            rg = tf.expand_dims(rg, -1)
            rg = tf.tile(rg, [1, int(indices.get_shape()[-1])])
            rg = tf.expand_dims(rg, -1)
            gidx = tf.concat([rg, idx], -1)
            return tf.gather_nd(values, gidx)
        
        _, register_s_last, register_h_last, memory_last, \
            read_weights_last, read_vec_last, write_weights_last, \
            usage_last, *temporal_states = states
        memory_last = K.reshape(memory_last,
            (-1,self.memory_size,self.word_size))
        read_weights_last = K.reshape(read_weights_last,
            (-1,self.read_head_count,self.memory_size))
        write_weights_last = K.reshape(write_weights_last,
            (-1,self.write_head_count,self.memory_size))
        if not self.enable_temporal:
            assert not temporal_states
        else:
            preced_last, link_last = temporal_states
            preced_last = K.reshape(preced_last,
                (-1,self.write_head_count,self.memory_size))
            link_last = K.reshape(link_last,
                (-1,self.write_head_count,self.memory_size,self.memory_size))
        
        # feeding controller with current input and last read vecs
        output = K.concatenate([inputs, read_vec_last, register_h_last])
        output = output @ self.kernel_controller_hidden
        output = output + self.bias_controller_hidden
        ctr_input_gate = output[:,:self.register_size*self.word_size]
        ctr_input_gate = K.sigmoid(ctr_input_gate)
        ctr_forget_gate = output[:,
            self.register_size*self.word_size:
            self.register_size*self.word_size*2]
        ctr_forget_gate = K.sigmoid(ctr_forget_gate)
        register_s = output[:,
            self.register_size*self.word_size*2:
            self.register_size*self.word_size*3]
        register_s = ctr_input_gate * K.tanh(register_s)
        register_s = register_s + ctr_forget_gate*register_s_last
        ctr_output_gate = output[:,
            self.register_size*self.word_size*3:
            self.register_size*self.word_size*4]
        ctr_output_gate = K.sigmoid(ctr_output_gate)
        register_h = ctr_output_gate * K.tanh(register_s)
        output = register_h @ self.kernel_controller_output
        
        # break down controller output into semi final output and interface vec
        interface_vec = output[:,self.output_size:]
        output = output[:,:self.output_size]
        if self.bypass_dropout_factor is not None:
            output = K.in_train_phase(K.dropout(
                output,self.bypass_dropout_factor),output,training=training)
        if self.enable_controller_output_resize:
            output = output @ self.kernel_controller_output_resize
        
        # break down interface vec
        interface_pos_last = 0
        interface_partition = []
        for interface_part_len in [
            self.read_head_count * self.word_size,
            self.read_head_count,
            self.write_head_count * self.word_size,
            self.write_head_count,
            self.write_head_count * self.word_size,
            self.write_head_count * self.word_size,
            self.read_head_count,
            self.write_head_count,
            self.write_head_count,
            *([self.read_head_count * 3] if self.enable_temporal else [])]:
            interface_pos = interface_pos_last + interface_part_len
            interface_partition.append(
                interface_vec[:,interface_pos_last:interface_pos])
            interface_pos_last = interface_pos
        read_keys, read_stre, write_keys, write_stre, erase_vecs, \
            write_vecs, free_gates, alloc_gates, write_gates, \
            *temporal_interface_partition = interface_partition
        read_keys = K.reshape(read_keys,(-1,self.read_head_count,self.word_size))
        read_stre = oneplus(read_stre)
        write_keys = K.reshape(write_keys,(-1,self.write_head_count,self.word_size))
        write_stre = oneplus(write_stre)
        erase_vecs = K.reshape(erase_vecs,(-1,self.write_head_count,self.word_size))
        erase_vecs = K.sigmoid(erase_vecs)
        write_vecs = K.reshape(write_vecs,(-1,self.write_head_count,self.word_size))
        free_gates = K.expand_dims(free_gates)
        free_gates = K.sigmoid(free_gates)
        alloc_gates = K.expand_dims(alloc_gates)
        alloc_gates = K.sigmoid(alloc_gates)
        write_gates = K.expand_dims(write_gates)
        write_gates = K.sigmoid(write_gates)
        if not self.enable_temporal:
            assert not temporal_interface_partition
        else:
            read_modes, = temporal_interface_partition
            read_modes = K.reshape(read_modes,(-1,self.read_head_count,3))
            read_modes = K.softmax(read_modes,axis=-1)
        
        # compute allocation vector
        retention = K.prod(1.-(free_gates*read_weights_last), axis=-2)
        # https://github.com/deepmind/dnc/blob/master/dnc/addressing.py
        # according to the deepmind implementation,
        # only write weight is not differentiable
        write_weights_last_nograd = K.stop_gradient(write_weights_last)
        # reduce for multi-write-head, not presented in original papaer
        write_weights_last_nograd = 1.-K.prod(1.-write_weights_last_nograd, axis=-2)
        usage = ((usage_last+write_weights_last_nograd) - \
                usage_last*write_weights_last_nograd) * retention
        # loop for multi-write-head support
        mwh_write_gates = write_gates * alloc_gates
        mwh_usage = usage
        mwh_alloc = []
        for i in range(self.write_head_count):
            # quickfix for tf.cumprod grad bug
            # https://github.com/tensorflow/tensorflow/issues/3862
            usage_qfixed = 1e-6 + (1-1e-6) * mwh_usage
            usage_asc,usage_perm = tf.nn.top_k(1.-usage_qfixed,k=self.memory_size)
            usage_asc = 1.-usage_asc
            alloc_asc = (1.-usage_asc) * tf.cumprod(usage_asc,axis=-1,exclusive=True)
            alloc_perm = batch_invert_permutation(usage_perm)
            alloc = batch_gather(alloc_asc, alloc_perm)
            mwh_alloc.append(alloc)
            mwh_usage += (1.-mwh_usage) * mwh_write_gates[:,i,:] * alloc
        alloc = K.stack(mwh_alloc, axis=-2)
        
        # compute write weight
        write_sims = similarity(memory_last,write_keys,write_stre)
        write_weights = write_gates*(alloc_gates*alloc + (1.-alloc_gates)*write_sims)
        
        # compute precedence and temporal links
        if self.enable_temporal:
            write_weights_rep_h =  K.expand_dims(write_weights,axis=-1)
            write_weights_rep_v =  K.expand_dims(write_weights,axis=-2)
            link = (1.-(write_weights_rep_h+write_weights_rep_v))*link_last + \
                write_weights_rep_h * K.expand_dims(preced_last,axis=-2)
            link = link * (1.-tf.eye(self.memory_size))
            read_weights_last_rep = K.repeat_elements(
                K.expand_dims(read_weights_last,axis=-3),self.write_head_count,-3)
            # reduce sum for multi-write-head
            link_forward = K.sum(read_weights_last_rep @ link,axis=-3)
            link_backward = K.sum(K.permute_dimensions(link @ K.permute_dimensions(
                read_weights_last_rep,(0,1,3,2)),(0,1,3,2)),axis=-3)
            preced = (1.-K.sum(
                write_weights,axis=-1,keepdims=True))*preced_last + write_weights
        
        # update memory
        m_reset = K.expand_dims(write_weights,axis=-1) @ \
            K.expand_dims(erase_vecs,axis=-2)
        # reduce prod for multi-write-head
        m_keep = K.prod(1.-m_reset,axis=-3)
        m_new = K.expand_dims(write_weights,axis=-1) @ \
            K.expand_dims(write_vecs,axis=-2)
        # reduce sum for multi-write-head
        m_new = K.sum(m_new, axis=-3)
        memory = memory_last*m_keep + m_new
        
        # compute read weights and read vectors
        read_sims = similarity(memory,read_keys,read_stre)
        if not self.enable_temporal:
            read_weights = read_sims
        else:
            read_weights = read_modes[:,:,0:1] * read_sims + \
                            read_modes[:,:,1:2] * link_forward + \
                            read_modes[:,:,2:3] * link_backward
        read_vecs = read_weights @ memory
        read_vec = K.reshape(read_vecs, (-1,self.read_head_count*self.word_size))
        
        # compute final output from controller output and current read vecs
        output = output + read_vec @ self.kernel_read_vec_to_output
        if self.enable_final_bias:
            output = output + self.bias_final_output
        
        return output, [
            output,
            register_s,
            register_h,
            K.reshape(memory, (-1,self.memory_size*self.word_size)),
            K.reshape(read_weights, (-1,self.read_head_count*self.memory_size)),
            read_vec,
            K.reshape(write_weights, (-1,self.write_head_count*self.memory_size)),
            usage,
            *([
                K.reshape(preced, (-1,self.write_head_count*self.memory_size)),
                K.reshape(link, (-1,self.write_head_count*self.memory_size**2))
            ] if self.enable_temporal else [])]
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [6]:
X = X_inputs = keras.layers.Input((None,), dtype='int32')
X = keras.layers.Embedding(data_input_dim, 16, mask_zero=True)(X)
X = keras.layers.RNN(DNCCell(data_output_dim, 8, 16), return_sequences=True)(X)
X = keras.layers.Lambda(lambda x,s: x[:,s:,:],
    arguments={'s':X_train.shape[1]//2})(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.TimeDistributed(keras.layers.Dense(data_output_dim))(X)
X = keras.layers.Softmax()(X)
X = [keras.layers.Lambda(lambda x,s: x[:,s,:], name=f'o{i+1}',
    arguments={'s':i}, output_shape=(data_output_dim,))(X)
    for i in range(X_train.shape[1]//2)]
M = keras.Model(X_inputs, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 16)     512         input_1[0][0]                    
__________________________________________________________________________________________________
rnn_1 (RNN)                     (None, None, 32)     5472        embedding_1[0][0]                
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 32)     0           rnn_1[0][0]                      
__________________________________________________________________________________________________
activation

In [7]:
M.fit(X_train, np.split(Y_train,Y_train.shape[-1],axis=-1),
    validation_data=(X_test,np.split(Y_test,Y_test.shape[-1],axis=-1)),
    batch_size=128, epochs=100, callbacks=[
    keras.callbacks.ReduceLROnPlateau('loss', patience=3, verbose=1)
])

Train on 10000 samples, validate on 1000 samples
Epoch 1/100
10000/10000 [==============================] - 50s 5ms/step - loss: 45.5194 - o1_loss: 3.0641 - o2_loss: 2.9940 - o3_loss: 2.9749 - o4_loss: 2.9605 - o5_loss: 2.9370 - o6_loss: 2.9062 - o7_loss: 2.8747 - o8_loss: 2.8581 - o9_loss: 2.8532 - o10_loss: 2.8483 - o11_loss: 2.8393 - o12_loss: 2.8315 - o13_loss: 2.7987 - o14_loss: 2.7349 - o15_loss: 2.6129 - o16_loss: 2.4308 - o1_acc: 0.0563 - o2_acc: 0.1044 - o3_acc: 0.1268 - o4_acc: 0.1268 - o5_acc: 0.1204 - o6_acc: 0.1105 - o7_acc: 0.1127 - o8_acc: 0.1207 - o9_acc: 0.1178 - o10_acc: 0.1099 - o11_acc: 0.0937 - o12_acc: 0.0887 - o13_acc: 0.0954 - o14_acc: 0.1195 - o15_acc: 0.1720 - o16_acc: 0.2704 - val_loss: 35.1935 - val_o1_loss: 2.0648 - val_o2_loss: 2.2009 - val_o3_loss: 2.3373 - val_o4_loss: 2.3607 - val_o5_loss: 2.2817 - val_o6_loss: 2.2210 - val_o7_loss: 2.1753 - val_o8_loss: 2.1431 - val_o9_loss: 2.1523 - val_o10_loss: 2.2095 - val_o11_loss: 2.2814 - val_o12_loss: 2.3942 - 

Epoch 7/100
10000/10000 [==============================] - 45s 5ms/step - loss: 26.0775 - o1_loss: 1.5058 - o2_loss: 1.7440 - o3_loss: 1.7866 - o4_loss: 1.7585 - o5_loss: 1.7270 - o6_loss: 1.6867 - o7_loss: 1.6242 - o8_loss: 1.5859 - o9_loss: 1.5984 - o10_loss: 1.6145 - o11_loss: 1.6137 - o12_loss: 1.6444 - o13_loss: 1.6416 - o14_loss: 1.6550 - o15_loss: 1.5728 - o16_loss: 1.3183 - o1_acc: 0.4188 - o2_acc: 0.3049 - o3_acc: 0.2834 - o4_acc: 0.2848 - o5_acc: 0.2996 - o6_acc: 0.3053 - o7_acc: 0.3354 - o8_acc: 0.3467 - o9_acc: 0.3478 - o10_acc: 0.3510 - o11_acc: 0.3424 - o12_acc: 0.3263 - o13_acc: 0.3322 - o14_acc: 0.3144 - o15_acc: 0.3290 - o16_acc: 0.4318 - val_loss: 26.1634 - val_o1_loss: 1.5283 - val_o2_loss: 1.7562 - val_o3_loss: 1.8011 - val_o4_loss: 1.8213 - val_o5_loss: 1.7538 - val_o6_loss: 1.7141 - val_o7_loss: 1.6982 - val_o8_loss: 1.6344 - val_o9_loss: 1.5780 - val_o10_loss: 1.6110 - val_o11_loss: 1.5332 - val_o12_loss: 1.6197 - val_o13_loss: 1.5928 - val_o14_loss: 1.6191 - val

Epoch 13/100
10000/10000 [==============================] - 46s 5ms/step - loss: 24.2018 - o1_loss: 1.4598 - o2_loss: 1.6982 - o3_loss: 1.7259 - o4_loss: 1.6785 - o5_loss: 1.6294 - o6_loss: 1.5616 - o7_loss: 1.4814 - o8_loss: 1.4221 - o9_loss: 1.4293 - o10_loss: 1.4392 - o11_loss: 1.4477 - o12_loss: 1.4871 - o13_loss: 1.4906 - o14_loss: 1.5210 - o15_loss: 1.4770 - o16_loss: 1.2530 - o1_acc: 0.4251 - o2_acc: 0.3188 - o3_acc: 0.3113 - o4_acc: 0.3120 - o5_acc: 0.3355 - o6_acc: 0.3528 - o7_acc: 0.3828 - o8_acc: 0.3918 - o9_acc: 0.3922 - o10_acc: 0.3851 - o11_acc: 0.3879 - o12_acc: 0.3728 - o13_acc: 0.3650 - o14_acc: 0.3620 - o15_acc: 0.3521 - o16_acc: 0.4378 - val_loss: 24.2638 - val_o1_loss: 1.4815 - val_o2_loss: 1.7233 - val_o3_loss: 1.7156 - val_o4_loss: 1.7068 - val_o5_loss: 1.6392 - val_o6_loss: 1.5804 - val_o7_loss: 1.4900 - val_o8_loss: 1.4565 - val_o9_loss: 1.3890 - val_o10_loss: 1.4395 - val_o11_loss: 1.4261 - val_o12_loss: 1.4927 - val_o13_loss: 1.5062 - val_o14_loss: 1.5377 - va

Epoch 19/100
10000/10000 [==============================] - 46s 5ms/step - loss: 22.7820 - o1_loss: 1.4461 - o2_loss: 1.6846 - o3_loss: 1.6837 - o4_loss: 1.6194 - o5_loss: 1.5531 - o6_loss: 1.4591 - o7_loss: 1.3791 - o8_loss: 1.3162 - o9_loss: 1.3123 - o10_loss: 1.3184 - o11_loss: 1.3218 - o12_loss: 1.3429 - o13_loss: 1.3622 - o14_loss: 1.4014 - o15_loss: 1.3925 - o16_loss: 1.1890 - o1_acc: 0.4287 - o2_acc: 0.3213 - o3_acc: 0.3253 - o4_acc: 0.3306 - o5_acc: 0.3589 - o6_acc: 0.3803 - o7_acc: 0.4012 - o8_acc: 0.4252 - o9_acc: 0.4341 - o10_acc: 0.4207 - o11_acc: 0.4279 - o12_acc: 0.4174 - o13_acc: 0.4075 - o14_acc: 0.3944 - o15_acc: 0.3808 - o16_acc: 0.4511 - val_loss: 22.9817 - val_o1_loss: 1.4544 - val_o2_loss: 1.6938 - val_o3_loss: 1.6505 - val_o4_loss: 1.6317 - val_o5_loss: 1.5367 - val_o6_loss: 1.4580 - val_o7_loss: 1.4119 - val_o8_loss: 1.3459 - val_o9_loss: 1.3170 - val_o10_loss: 1.4066 - val_o11_loss: 1.3668 - val_o12_loss: 1.4106 - val_o13_loss: 1.3821 - val_o14_loss: 1.4211 - va

Epoch 25/100
10000/10000 [==============================] - 45s 5ms/step - loss: 21.4561 - o1_loss: 1.4396 - o2_loss: 1.6660 - o3_loss: 1.6449 - o4_loss: 1.5638 - o5_loss: 1.4728 - o6_loss: 1.3675 - o7_loss: 1.2716 - o8_loss: 1.2148 - o9_loss: 1.2001 - o10_loss: 1.1963 - o11_loss: 1.2114 - o12_loss: 1.2273 - o13_loss: 1.2443 - o14_loss: 1.2793 - o15_loss: 1.3050 - o16_loss: 1.1514 - o1_acc: 0.4282 - o2_acc: 0.3212 - o3_acc: 0.3323 - o4_acc: 0.3501 - o5_acc: 0.3821 - o6_acc: 0.4115 - o7_acc: 0.4517 - o8_acc: 0.4652 - o9_acc: 0.4715 - o10_acc: 0.4735 - o11_acc: 0.4609 - o12_acc: 0.4600 - o13_acc: 0.4521 - o14_acc: 0.4395 - o15_acc: 0.4104 - o16_acc: 0.4663 - val_loss: 21.5553 - val_o1_loss: 1.4498 - val_o2_loss: 1.6776 - val_o3_loss: 1.6099 - val_o4_loss: 1.5732 - val_o5_loss: 1.4552 - val_o6_loss: 1.3700 - val_o7_loss: 1.2756 - val_o8_loss: 1.2348 - val_o9_loss: 1.1697 - val_o10_loss: 1.2108 - val_o11_loss: 1.2071 - val_o12_loss: 1.2559 - val_o13_loss: 1.2882 - val_o14_loss: 1.2891 - va

...

Epoch 73/100
10000/10000 [==============================] - 45s 5ms/step - loss: 14.3182 - o1_loss: 1.1846 - o2_loss: 1.3744 - o3_loss: 1.3533 - o4_loss: 1.2693 - o5_loss: 1.1179 - o6_loss: 0.9753 - o7_loss: 0.8520 - o8_loss: 0.7583 - o9_loss: 0.6869 - o10_loss: 0.6460 - o11_loss: 0.6295 - o12_loss: 0.6289 - o13_loss: 0.6386 - o14_loss: 0.6918 - o15_loss: 0.7490 - o16_loss: 0.7624 - o1_acc: 0.4815 - o2_acc: 0.4016 - o3_acc: 0.4190 - o4_acc: 0.4490 - o5_acc: 0.5036 - o6_acc: 0.5686 - o7_acc: 0.6273 - o8_acc: 0.6667 - o9_acc: 0.7069 - o10_acc: 0.7254 - o11_acc: 0.7448 - o12_acc: 0.7433 - o13_acc: 0.7414 - o14_acc: 0.7138 - o15_acc: 0.6764 - o16_acc: 0.6497 - val_loss: 14.1306 - val_o1_loss: 1.2067 - val_o2_loss: 1.3795 - val_o3_loss: 1.3704 - val_o4_loss: 1.2701 - val_o5_loss: 1.1380 - val_o6_loss: 1.0212 - val_o7_loss: 0.8825 - val_o8_loss: 0.7981 - val_o9_loss: 0.6617 - val_o10_loss: 0.6385 - val_o11_loss: 0.5768 - val_o12_loss: 0.5299 - val_o13_loss: 0.5599 - val_o14_loss: 0.6505 - va

Epoch 79/100
10000/10000 [==============================] - 46s 5ms/step - loss: 12.9920 - o1_loss: 1.1534 - o2_loss: 1.3320 - o3_loss: 1.3183 - o4_loss: 1.2365 - o5_loss: 1.0832 - o6_loss: 0.9189 - o7_loss: 0.7916 - o8_loss: 0.6928 - o9_loss: 0.6041 - o10_loss: 0.5463 - o11_loss: 0.5028 - o12_loss: 0.4893 - o13_loss: 0.4914 - o14_loss: 0.5469 - o15_loss: 0.6310 - o16_loss: 0.6535 - o1_acc: 0.4910 - o2_acc: 0.4264 - o3_acc: 0.4371 - o4_acc: 0.4588 - o5_acc: 0.5181 - o6_acc: 0.5986 - o7_acc: 0.6552 - o8_acc: 0.7021 - o9_acc: 0.7451 - o10_acc: 0.7749 - o11_acc: 0.8036 - o12_acc: 0.8025 - o13_acc: 0.8046 - o14_acc: 0.7783 - o15_acc: 0.7342 - o16_acc: 0.7160 - val_loss: 13.3204 - val_o1_loss: 1.1964 - val_o2_loss: 1.3480 - val_o3_loss: 1.3612 - val_o4_loss: 1.2408 - val_o5_loss: 1.0747 - val_o6_loss: 0.9433 - val_o7_loss: 0.8227 - val_o8_loss: 0.7061 - val_o9_loss: 0.5935 - val_o10_loss: 0.6018 - val_o11_loss: 0.5134 - val_o12_loss: 0.4753 - val_o13_loss: 0.5090 - val_o14_loss: 0.6133 - va

Epoch 85/100
10000/10000 [==============================] - 45s 4ms/step - loss: 12.6464 - o1_loss: 1.1163 - o2_loss: 1.2926 - o3_loss: 1.2833 - o4_loss: 1.2124 - o5_loss: 1.0578 - o6_loss: 0.9021 - o7_loss: 0.7800 - o8_loss: 0.6887 - o9_loss: 0.6061 - o10_loss: 0.5512 - o11_loss: 0.5063 - o12_loss: 0.4799 - o13_loss: 0.4664 - o14_loss: 0.5169 - o15_loss: 0.5808 - o16_loss: 0.6055 - o1_acc: 0.5080 - o2_acc: 0.4386 - o3_acc: 0.4504 - o4_acc: 0.4731 - o5_acc: 0.5326 - o6_acc: 0.6076 - o7_acc: 0.6618 - o8_acc: 0.7031 - o9_acc: 0.7432 - o10_acc: 0.7663 - o11_acc: 0.7912 - o12_acc: 0.8051 - o13_acc: 0.8128 - o14_acc: 0.7894 - o15_acc: 0.7572 - o16_acc: 0.7389 - val_loss: 12.8010 - val_o1_loss: 1.1508 - val_o2_loss: 1.2828 - val_o3_loss: 1.3042 - val_o4_loss: 1.1879 - val_o5_loss: 1.0570 - val_o6_loss: 0.9376 - val_o7_loss: 0.8258 - val_o8_loss: 0.7014 - val_o9_loss: 0.5931 - val_o10_loss: 0.5810 - val_o11_loss: 0.4917 - val_o12_loss: 0.4376 - val_o13_loss: 0.4644 - val_o14_loss: 0.5754 - va

Epoch 91/100
10000/10000 [==============================] - 45s 5ms/step - loss: 11.9712 - o1_loss: 1.0930 - o2_loss: 1.2567 - o3_loss: 1.2596 - o4_loss: 1.1962 - o5_loss: 1.0538 - o6_loss: 0.8939 - o7_loss: 0.7501 - o8_loss: 0.6459 - o9_loss: 0.5420 - o10_loss: 0.4742 - o11_loss: 0.4374 - o12_loss: 0.4152 - o13_loss: 0.4144 - o14_loss: 0.4712 - o15_loss: 0.5241 - o16_loss: 0.5436 - o1_acc: 0.5147 - o2_acc: 0.4519 - o3_acc: 0.4635 - o4_acc: 0.4786 - o5_acc: 0.5314 - o6_acc: 0.6034 - o7_acc: 0.6801 - o8_acc: 0.7238 - o9_acc: 0.7775 - o10_acc: 0.8067 - o11_acc: 0.8317 - o12_acc: 0.8375 - o13_acc: 0.8379 - o14_acc: 0.8151 - o15_acc: 0.7864 - o16_acc: 0.7746 - val_loss: 12.4843 - val_o1_loss: 1.1239 - val_o2_loss: 1.2610 - val_o3_loss: 1.2844 - val_o4_loss: 1.2117 - val_o5_loss: 1.0792 - val_o6_loss: 0.9772 - val_o7_loss: 0.8194 - val_o8_loss: 0.7187 - val_o9_loss: 0.5899 - val_o10_loss: 0.5409 - val_o11_loss: 0.4941 - val_o12_loss: 0.4322 - val_o13_loss: 0.4268 - val_o14_loss: 0.5055 - va

Epoch 97/100
10000/10000 [==============================] - 45s 5ms/step - loss: 11.1355 - o1_loss: 1.0633 - o2_loss: 1.2225 - o3_loss: 1.2229 - o4_loss: 1.1668 - o5_loss: 1.0218 - o6_loss: 0.8555 - o7_loss: 0.7128 - o8_loss: 0.5958 - o9_loss: 0.4977 - o10_loss: 0.4312 - o11_loss: 0.3873 - o12_loss: 0.3583 - o13_loss: 0.3518 - o14_loss: 0.3755 - o15_loss: 0.4234 - o16_loss: 0.4487 - o1_acc: 0.5280 - o2_acc: 0.4663 - o3_acc: 0.4767 - o4_acc: 0.4899 - o5_acc: 0.5474 - o6_acc: 0.6213 - o7_acc: 0.6906 - o8_acc: 0.7479 - o9_acc: 0.8000 - o10_acc: 0.8270 - o11_acc: 0.8537 - o12_acc: 0.8651 - o13_acc: 0.8716 - o14_acc: 0.8573 - o15_acc: 0.8404 - o16_acc: 0.8211 - val_loss: 11.5255 - val_o1_loss: 1.1098 - val_o2_loss: 1.2355 - val_o3_loss: 1.2598 - val_o4_loss: 1.1766 - val_o5_loss: 1.0237 - val_o6_loss: 0.8822 - val_o7_loss: 0.7331 - val_o8_loss: 0.6118 - val_o9_loss: 0.5012 - val_o10_loss: 0.5083 - val_o11_loss: 0.4369 - val_o12_loss: 0.3971 - val_o13_loss: 0.3724 - val_o14_loss: 0.4039 - va